<a href="https://colab.research.google.com/github/Alphy-Ototo/Alphy_Codes/blob/main/labs/de_lab_1__Ototo_Alphonsinah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Engineering - Lab 1


## Intro to table of contents

In this lab we'll be introduced to the Google Colab environment. To start this notebook simply go to "Runtime" and choose "Run All". This will connect Colab to your virtual machine and run the code needed to get your database setup.

You'll notice that notebooks are setup with a table of contents navigation bar on the left nav bar of your screen. Mouse over and hover to see which options are available and click "Table of Contents" to see the major sections to this Notebook.  

## Database setups (do not modify)

In [4]:
# setups including getting the database

!pip -q install --upgrade ipython
!pip -q install --upgrade ipython-sql

!wget -O northwind.db https://github.com/matthewpecsok/data_engineering/raw/main/data/northwind.db

import sqlite3
con = sqlite3.connect("northwind.db")

%load_ext sql
%sql sqlite:///northwind.db

%config SqlMagic.style = '_DEPRECATED_DEFAULT'
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.5/825.5 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.32.0 which is incompatible.
--2025-02-20 03:19:18--  https://github.com/matthewpecsok/data_engineering/raw/main/data/northwind.db
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/matthewpecsok/data_engineering/main/data/northwind.db [following]
--2025-02-20 03:19:18--  https://raw.githubusercontent.com/matthewpecsok/data_engineering/main/data/northwind.d

## ERD of the Northwind Database


![ERD](https://github.com/matthewpecsok/data_engineering/blob/main/data/Northwind_ERD.png?raw=true)


Using %sql allows for one line sql statements
Using %%sql allows for multi-line sql statements and better readability for long sql queries.

In [5]:
%sql select * from products limit 4

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22,53,0,0,0


In [6]:
%%sql

select
count(1) as product_count
from
products

,product_count
0,77


## Lab question 1

In [10]:
%%sql

select
count(*) as unitprice_gt_18
from
products
where UnitPrice > 18;

,unitprice_gt_18
0,43


## Lab question 2


In [13]:
 %%sql

   select c.CompanyName,
   count(o.OrderID) as order_count
 from
   customers c
 join
   orders o on c.CustomerID = o.CustomerID
 group by
   c.CompanyName
 order by
   order_count desc
 limit 5;




,CompanyName,order_count
0,Save-a-lot Markets,31
1,Ernst Handel,30
2,QUICK-Stop,28
3,Hungry Owl All-Night Grocers,19
4,Folk och fä HB,19


## Lab question 3



In [35]:
 %%sql

  select
    o.OrderID,
    c.CategoryName
  from
    "order details" od
  join
    products p on od.ProductID = p.ProductID
  join
    categories c on p.CategoryID = c.CategoryID
  join
    orders o on od.OrderID = o.OrderID
  group by
    c.CategoryID
  limit 5;


,OrderID,CategoryName
0,10253,Beverages
1,10250,Condiments
2,10252,Confections
3,10248,Dairy Products
4,10248,Grains/Cereals


## Lab question 4



In [19]:
 %%sql

  select
     count(o.OrderID) as RegionCount, ShipRegion
  from
    orders o
  group by
    ShipRegion
  having
    count(o.OrderID) > 100
  order by
    RegionCount desc;



,RegionCount,ShipRegion
0,276,Western Europe
1,152,North America
2,145,South America


## Lab Question 5

In [20]:
 %%sql

  select
    max(UnitPrice) as MaxPrice,
    min(UnitPrice) as MinPrice,
    round(avg(UnitPrice),2) as average_unit_price
  from
    products;

,MaxPrice,MinPrice,average_unit_price
0,263.5,2.5,28.87


## Lab Question 6

In [24]:
%%sql
     select
       (od.UnitPrice * od.Quantity) as original_total,
       (od.UnitPrice * od.Quantity) * (1 - od.Discount) as discounted_total,
       (od.UnitPrice * od.Quantity * od.Discount) as discount_amount,
       od.Discount as discount_percentage,
       c.CompanyName
     from
       "order details" od
     join
       orders o on od.OrderID = o.ORDERID
     join
       customers c on o.CustomerID = c.CustomerID
     where
       c.CompanyName = 'Ernst Handel'
       and (od.UnitPrice * od.Quantity) > 4000;

,original_total,discounted_total,discount_amount,discount_percentage,CompanyName
0,4216.0,4005.2,210.8,0.05,Ernst Handel
1,6360.0,6042.0,318.0,0.05,Ernst Handel
2,6050.0,6050.0,0.0,0.00,Ernst Handel
3,4322.5,4322.5,0.0,0.00,Ernst Handel


## Lab Question 7

In [25]:
 %%sql

   select
     strftime('%Y', OrderDate) AS order_year,
     COUNT(OrderID) AS order_count
   from
     orders
   group by
     order_year
   order by
     order_year;

,order_year,order_count
0,2016,152
1,2017,408
2,2018,270


## Lab Question 8

In [28]:
 %%sql

   select
       e.FirstName || ' ' || e.LastName as EmployeeName,
       count(o.OrderID) as order_count
   from
      Employees e
   join
      Orders o on e.EmployeeID = o.EmployeeID
   group by
      EmployeeName
   order by
      order_count desc
   limit 5;


,EmployeeName,order_count
0,Margaret Peacock,156
1,Janet Leverling,127
2,Nancy Davolio,123
3,Laura Callahan,104
4,Andrew Fuller,96


## Lab Question 9

In [29]:
 %%sql

   select count(*)
   from sqlite_master
   where type = 'table';


,count(*)
0,14


## Lab Question 10


In [30]:
%%sql

   select count(*) as view_count
   from sqlite_master
   where type = 'view';

,view_count
0,17


## Lab Question 11

In [36]:
%%sql

   select count(*) as left_count
   from customers c
   left join orders o on c.CustomerID = o.CustomerID

,left_count
0,834


## Lab Question 12

In [37]:
%%sql

   select count(*) as inner_count
   from customers c
   inner join orders o on c.CustomerID = o.CustomerID

,inner_count
0,830


## Lab Question 13

In [52]:
 %%sql
    Drop table if exists NewTable;

""


In [53]:
 %%sql
    create table NewTable (
     Name text not null,
     Age integer not null,
     Address text
);

""


In [54]:
%%sql
   select count(*) as row_count from NewTable;

,row_count
0,0


## Lab Question 14

In [55]:
%%sql
   insert into NewTable (Name, Age) values
          ('Brian', 24),
          ('Frank', 22),
          ('Ignatious', 14),
          ('Yvonne', 20),
          ('Jimmy', 15);

""


## Lab Question 15

In [56]:
%%sql
  select *
  from NewTable

,Name,Age,Address
0,Brian,24,None
1,Frank,22,None
2,Ignatious,14,None
3,Yvonne,20,None
4,Jimmy,15,None


In [ ]:
# replace ###### with your file name
# make sure you have your google drive mounted.

!cp "/content/drive/MyDrive/Colab Notebooks/######.ipynb" ./
!jupyter nbconvert --to html "######.ipynb"